In [ ]:
%pip install geopy
%pip install kagglehub

import kagglehub
import numpy as np
import pandas as pd
import pytz

from datetime import datetime
from geopy import distance
from scipy.stats import zscore

nifc_data = pd.read_csv('nifc_wildfires.csv', delimiter='\t')
noaa_data = pd.read_csv('noaa_wildfires.csv', skiprows=3)
nifc_human_acres = pd.read_csv('nifc_human_caused_acres.csv', delimiter='\t')
nifc_lightning_acres = pd.read_csv('nifc_lightning_caused_acres.csv', delimiter='\t')
wildfire_noaa_data = pd.read_csv('noaa_wildfires_sylvia.csv')

path = kagglehub.dataset_download("sobhanmoosavi/us-weather-events")
weather_events = pd.read_csv(path + '/WeatherEvents_Jan2016-Dec2022.csv')
path_2 = kagglehub.dataset_download("robikscube/zillow-home-value-index")
zhvi = pd.read_csv(path_2 + '/ZHVI.csv')

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# `wildfire-hard-5`: On average, how many more annual fires are reported by NOAA compared to NIFC since 2000? Round to the nearest whole number.

In [2]:
noaa_data['Datetime'] = pd.to_datetime(noaa_data['Date'], format='%Y%m')
noaa_data['Year'] = noaa_data['Datetime'].dt.year
noaa_data['Month'] = noaa_data['Datetime'].dt.month
noaa_yearly_data = noaa_data.groupby(['Year']).agg({'Number of Fires': 'sum', 'Acres Burned': 'sum'}).reset_index()

In [3]:
# Perform an inner join on the 'Year' column
merged_data = pd.merge(
    nifc_data.rename(columns={'Fires': 'NIFC_Fires', 'Acres': 'NIFC_Acres'}),
    noaa_yearly_data.rename(columns={'Number of Fires': 'NOAA_Number_of_Fires', 'Acres Burned': 'NOAA_Acres_Burned'}),
    on='Year',
    how='inner'
)
merged_data['Difference_in_Fires'] = merged_data['NOAA_Number_of_Fires'] - merged_data['NIFC_Fires'].str.replace(',', '').astype(int)
round(merged_data['Difference_in_Fires'].mean())

-1039

# `wildfire-hard-6`: What is the correlation between (1) the difference between the number of NOAA and NIFC-reported fires and (2) the difference between the acres burned by NOAA and NIFC-reported fires, on an annual basis?
Comment: Make sure to mention we need to round to 3 decimal places.

In [4]:
merged_data['Difference_in_Acres'] = merged_data['NOAA_Acres_Burned'] - merged_data['NIFC_Acres'].str.replace(',', '').str.replace('*', '').astype(int)
round(merged_data['Difference_in_Fires'].corr(merged_data['Difference_in_Acres']), 3)

/var/folders/mt/lv8dl5zj78n8fw343s4t1jmc0000gn/T/ipykernel_70624/1941409489.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  merged_data['Difference_in_Acres'] = merged_data['NOAA_Acres_Burned'] - merged_data['NIFC_Acres'].str.replace(',', '').str.replace('*', '').astype(int)


0.519

# `wildfire-hard-7`: Which geographic area had the most anomalous year (by Z-score) in terms of total acres burned compared to its historical annual average, and what year was it in?
Comment: The answer is different!

In [5]:
nifc_lightning_acres.rename(columns={'Western Great Basin*': 'Western Great Basin'}, inplace=True)
nifc_human_acres.iloc[:, 1:] = nifc_human_acres.iloc[:, 1:].replace({',': ''}, regex=True).astype(float).astype('Int64')
nifc_lightning_acres.iloc[:, 1:] = nifc_lightning_acres.iloc[:, 1:].replace({',': ''}, regex=True).astype(float).astype('Int64')
combined_acres = pd.merge(nifc_human_acres, nifc_lightning_acres, on='Year', suffixes=('_human', '_lightning'))

regions = nifc_human_acres.columns[1:]
for region in regions:
    combined_acres[region] = combined_acres[f"{region}_human"] + combined_acres[f"{region}_lightning"]
combined_acres.drop(columns=[f"{region}_human" for region in regions] + [f"{region}_lightning" for region in regions], inplace=True)

/var/folders/mt/lv8dl5zj78n8fw343s4t1jmc0000gn/T/ipykernel_70624/3805387751.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  nifc_human_acres.iloc[:, 1:] = nifc_human_acres.iloc[:, 1:].replace({',': ''}, regex=True).astype(float).astype('Int64')
/var/folders/mt/lv8dl5zj78n8fw343s4t1jmc0000gn/T/ipykernel_70624/3805387751.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  nifc_lightning_acres.iloc[:, 1:] = nifc_lightning_acres.iloc[:, 1:].replace({',': ''}, regex=True).astype(float).astype('Int64')


In [6]:
z_scores = combined_acres.copy()
for region in regions:
    z_scores[region] = zscore(combined_acres[region].fillna(0).astype(int), nan_policy='omit')

In [7]:
max_value = z_scores[regions].to_numpy().max()
row, col = np.where(z_scores[regions].to_numpy() == max_value)
region = regions[col[0]]
year = z_scores.iloc[row]['Year'].values[0]
print(f"Max Z-score: {max_value} in {region} for year {year}")

Max Z-score: 3.9498889601744533 in Northwest for year 2020


# `wildfire-15`: In 2016, what percentage of fires were brought under control with it raining moderately or heavily (>0.05 in) in the fire area on the same or previous day? Give an upper bound assuming that the narrowest diameter of the fire region is 1km.
Comment: I'd like clarification that 'on the same or previous day' refers to the day the fire was controlled. Perhaps 'on or a day before the control day' could do it.

In [8]:
# first filter is for types of precipitation, the year filter needs to come after splitting out UTC times, and the fire area / timing needs to come after the fire dataset is processed
mask =  (weather_events['Precipitation(in)'] > 0.05) & (
            (weather_events['Type'] == 'Precipitation') |
            (weather_events['Type'] == 'Rain') # | weather_events['Type'] == 'Storm') # according to the paper, storm means strong winds
        ) & (
            (weather_events['Severity'] == 'Moderate') |
            (weather_events['Severity'] == 'Heavy') |
            (weather_events['Severity'] == 'Severe')
        )
initial_filter_weather_events = weather_events[mask]

In [9]:
# Convert UTC start and end times to local time zone
def convert_to_local_time(utc_time, time_zone):
    utc_datetime = datetime.strptime(utc_time, '%Y-%m-%d %H:%M:%S')
    tz = pytz.timezone(time_zone)
    return tz.normalize(pytz.utc.localize(utc_datetime).astimezone(tz))
# convert_to_local_time('2016-02-01 08:15:00', 'US/Mountain').date()

initial_filter_weather_events['StartTime(Local)'] = initial_filter_weather_events.apply(
    lambda row: convert_to_local_time(row['StartTime(UTC)'], row['TimeZone']), axis=1
)
initial_filter_weather_events['EndTime(Local)'] = initial_filter_weather_events.apply(
    lambda row: convert_to_local_time(row['EndTime(UTC)'], row['TimeZone']), axis=1
)

initial_filter_weather_events['StartDay(Local)'] = initial_filter_weather_events['StartTime(Local)'].apply(lambda x: x.day)
initial_filter_weather_events['EndDay(Local)'] = initial_filter_weather_events['EndTime(Local)'].apply(lambda x: x.day)

initial_filter_weather_events['StartDayOfYear(Local)'] = initial_filter_weather_events['StartTime(Local)'].apply(lambda x: x.timetuple().tm_yday)
initial_filter_weather_events['EndDayOfYear(Local)'] = initial_filter_weather_events['EndTime(Local)'].apply(lambda x: x.timetuple().tm_yday)

initial_filter_weather_events['StartYear(Local)'] = initial_filter_weather_events['StartTime(Local)'].apply(lambda x: x.year)
initial_filter_weather_events['EndYear(Local)'] = initial_filter_weather_events['EndTime(Local)'].apply(lambda x: x.year)
initial_filter_weather_events['SameYear'] = initial_filter_weather_events['StartYear(Local)'] == initial_filter_weather_events['EndYear(Local)']

# seems like there are actually some weather events that start and end in different years, wonder if they will stay around after we join with fire incidents; seems safe to filter by start year
initial_filter_weather_events[initial_filter_weather_events['SameYear'] == False].groupby(['StartYear(Local)', 'EndYear(Local)']).size().reset_index(name='Counts')


/var/folders/mt/lv8dl5zj78n8fw343s4t1jmc0000gn/T/ipykernel_70624/3039365494.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  initial_filter_weather_events['StartTime(Local)'] = initial_filter_weather_events.apply(
/var/folders/mt/lv8dl5zj78n8fw343s4t1jmc0000gn/T/ipykernel_70624/3039365494.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  initial_filter_weather_events['EndTime(Local)'] = initial_filter_weather_events.apply(
/var/folders/mt/lv8dl5zj78n8fw343s4t1jmc0000gn/T/ipykernel_70624/3039365494.py:

,StartYear(Local),EndYear(Local),Counts
0,2016,2017,23
1,2018,2019,54
2,2019,2020,2
3,2020,2021,45
4,2021,2022,16


In [10]:
filtered_weather_events = initial_filter_weather_events[initial_filter_weather_events['StartYear(Local)'] == 2016]

# meanwhile, it seems like the noaa data does not have 2016 fires that cross into 2015 or 2017
wildfire_noaa_data.groupby(['start_year', 'control_year']).size().reset_index(name='Counts')

,start_year,control_year,Counts
0,2002,2002,403
1,2003,2003,465
2,2004,2004,273
3,2005,2005,515
4,2006,2006,817
5,2007,2007,600
6,2008,2008,473
7,2009,2009,419
8,2010,2010,333
9,2011,2011,565


In [11]:
initial_filter_fires = wildfire_noaa_data[(wildfire_noaa_data['start_year'] == 2016)]
def find_fire_radius(latitude, longitude, hectares):
    area = hectares / 100
    a = area / 1 # assuming minimum diameter of 1km
    return max(a, 0.5)  # min radius of 0.5km

initial_filter_fires['upper_bound_fire_radius_km'] = initial_filter_fires.apply(
    lambda row: find_fire_radius(row['latitude'], row['longitude'], row['hec']),
    axis=1
)

/var/folders/mt/lv8dl5zj78n8fw343s4t1jmc0000gn/T/ipykernel_70624/3785908159.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  initial_filter_fires['upper_bound_fire_radius_km'] = initial_filter_fires.apply(


In [12]:
# Filter weather events that overlap with the control day or the day before the control day of the fire
def find_overlapping_weather_events(fire_row, weather_events):
    control_day = fire_row.get('control_day_of_year')
    control_year = fire_row.get('control_year')
    fitting_weather_events = weather_events[
        (weather_events['StartDayOfYear(Local)'] <= control_day) &
        (weather_events['EndDayOfYear(Local)'] >= control_day - 1) &
        (weather_events['StartYear(Local)'] == control_year)
    ]

    fire_lat, fire_long, fire_radius_km = fire_row.get('latitude'), fire_row.get('longitude'), fire_row.get('upper_bound_fire_radius_km')

    if fitting_weather_events.empty:
        return False

    closest_weather_event_distance = float('inf')

    for _, weather_event in fitting_weather_events.iterrows():
        weather_lat, weather_long = weather_event['LocationLat'], weather_event['LocationLng']

        # haversine formula to convert lat/long to km
        # https://en.wikipedia.org/wiki/Haversine_formula
        R = 6371.001 # arithmetic mean of the earth's radius in km https://en.wikipedia.org/wiki/Earth_radius
        lat1, long1, lat2, long2 = map(np.radians, [fire_lat, fire_long, weather_lat, weather_long])
        d_lat = lat2 - lat1
        d_lon = long2 - long1
        weather_distance = 2 * R * np.arcsin(np.sqrt((np.sin(d_lat / 2) ** 2) + np.cos(lat1) * np.cos(lat2) * (np.sin(d_lon / 2) ** 2)))
        # a = np.sin(d_lat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(d_lon / 2)**2
        # c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
        # weather_distance = R * c
        # weather_distance = distance.distance((fire_lat, fire_long), (weather_lat, weather_long)).km

        if weather_distance < closest_weather_event_distance:
            closest_weather_event_distance = weather_distance
    
    if closest_weather_event_distance > fire_radius_km:
        return False
    else:
        return closest_weather_event_distance <= fire_radius_km

initial_filter_fires['OverlappingWeatherEvents'] = initial_filter_fires.apply(
    lambda row: find_overlapping_weather_events(row, filtered_weather_events), axis=1
)

# Display the result
initial_filter_fires.groupby(['OverlappingWeatherEvents']).size().reset_index(name='Counts')

/var/folders/mt/lv8dl5zj78n8fw343s4t1jmc0000gn/T/ipykernel_70624/3640368283.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  initial_filter_fires['OverlappingWeatherEvents'] = initial_filter_fires.apply(


,OverlappingWeatherEvents,Counts
0,False,472
1,True,7


In [13]:
print(str(initial_filter_fires['OverlappingWeatherEvents'].mean() * 100) + '%')

1.4613778705636742%


# `wildfire-16`: According to NOAA, what percentage of wildfires account for 90% of residential houses damaged in 2008?
Comment: Unless we want exactly 90%, so I would suggest rephrasing for 'smallest percentage' and 'at least 90% of'. Otherwise, rounding could save us here.

In [14]:
yearly_wildfire_noaa = wildfire_noaa_data.groupby('start_year').agg({
    'prim_threatened_aggregate': 'sum'
}).reset_index()
residential_proportion_threshold = yearly_wildfire_noaa[yearly_wildfire_noaa['start_year'] == 2008]['prim_threatened_aggregate'].sum() * 0.9
total_count = len(wildfire_noaa_data[wildfire_noaa_data['start_year'] == 2008])

In [15]:
fires_sorted_by_prim_threat = wildfire_noaa_data[wildfire_noaa_data['start_year'] == 2008].sort_values(by='prim_threatened_aggregate', ascending=False)['prim_threatened_aggregate']

# smallest number of fires to cross the threshold, when sorted from most damaging to least
cumulative_sum = 0
cumulative_count = 0
for fire in fires_sorted_by_prim_threat:
    cumulative_sum += fire
    cumulative_count += 1
    if int(cumulative_sum + fire) >= int(residential_proportion_threshold):
        break
    
str(round(cumulative_count / total_count * 100, 4)) + "% of fires"

'4.4397% of fires'

In [16]:
# the number of fires exactly needed to cross the threshold
cumulative_sum = 0
cumulative_count = 0
for fire in fires_sorted_by_prim_threat:
    if int(cumulative_sum + fire) == int(residential_proportion_threshold):
        break
    if int(cumulative_sum + fire) > int(residential_proportion_threshold):
        continue
    cumulative_sum += fire
    cumulative_count += 1

str(round(cumulative_count / total_count * 100, 4)) + "% of fires"

'4.6512% of fires'

# `wildfire-17`: Based on NOAA data, what are the top 3 states that lost the most residential property in value between 2005 and 2010 (including 2005 and 2010)? Answer in state full names.
Comment: The estimated residential value of each house impacted by a fire can vary depending on how we choose the month of the relevant average residential value.

In [17]:
zhvi['Datetime'] = pd.to_datetime(zhvi['Unnamed: 0'])
zhvi['Year'] = zhvi['Datetime'].dt.year
zhvi['Month'] = zhvi['Datetime'].dt.month
zhvi.drop(columns=['Unnamed: 0'], inplace=True)

state_name_to_abbreviation = {
    "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR", "California": "CA",
    "Colorado": "CO", "Connecticut": "CT", "Delaware": "DE", "Florida": "FL", "Georgia": "GA",
    "Hawaii": "HI", "Idaho": "ID", "Illinois": "IL", "Indiana": "IN", "Iowa": "IA",
    "Kansas": "KS", "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME", "Maryland": "MD",
    "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN", "Mississippi": "MS", "Missouri": "MO",
    "Montana": "MT", "Nebraska": "NE", "Nevada": "NV", "New Hampshire": "NH", "New Jersey": "NJ",
    "New Mexico": "NM", "New York": "NY", "North Carolina": "NC", "North Dakota": "ND", "Ohio": "OH",
    "Oklahoma": "OK", "Oregon": "OR", "Pennsylvania": "PA", "Rhode Island": "RI", "South Carolina": "SC",
    "South Dakota": "SD", "Tennessee": "TN", "Texas": "TX", "Utah": "UT", "Vermont": "VT",
    "Virginia": "VA", "Washington": "WA", "West Virginia": "WV", "Wisconsin": "WI", "Wyoming": "WY",
    "the District of Columbia": "DC",
}
abbreviation_to_state_name = {}
for state, abbreviation in state_name_to_abbreviation.items():
    abbreviation_to_state_name[abbreviation] = state

wildfire_noaa_data['start_month'] = pd.to_datetime(wildfire_noaa_data['start_date']).dt.month
wildfire_noaa_data['control_month'] = pd.to_datetime(wildfire_noaa_data['controlled_date']).dt.month
noaa_data_masked = wildfire_noaa_data[(wildfire_noaa_data['start_year'] >= 2005) & (wildfire_noaa_data['start_year'] <= 2010)] # there are no wildfires that span multiple years so this is safe
# seems like there's multiple fires that span the same month though
len(noaa_data_masked[noaa_data_masked['start_month'] != noaa_data_masked['control_month']])

938

In [18]:
# fortunately there's no missing values to worry about in this time range
noaa_data_masked['start_date'].isna().sum(), noaa_data_masked['controlled_date'].isna().sum(), noaa_data_masked['state'].isna().sum(), noaa_data_masked['prim_threatened_aggregate'].isna().sum()

(0, 0, 0, 0)

In [19]:
# for each fire listed in noaa data masked, get the weighted home value for the duration of the fire, making the assumption that the value could be realized at any point in the fire duration if the fire didn't happen
def get_weighted_residential_value_for_fire(start_date, end_date, state, prim_threatened_aggregate): # can guarantee fields are populated
    # Get the start and end dates of the fire
    dt_start_date = pd.to_datetime(start_date)
    dt_end_date = pd.to_datetime(end_date)
    start_year = dt_start_date.year
    start_month = dt_start_date.month
    # get number of days in each month between start and end date
    num_days_per_month = pd.date_range(start=dt_start_date, end=dt_end_date, freq='M').day
    num_days_total = (dt_end_date - dt_start_date).days + 1
    if num_days_per_month.empty: # if the fire does not span multiple months
        num_days_per_month = [num_days_total]
    # get the zillow home value per month for the state
    home_value_per_month = [
        zhvi.loc[(zhvi['Year'] == start_year) & (zhvi['Month'] == start_month + i), state].values[0]
        if not zhvi.loc[(zhvi['Year'] == start_year) & (zhvi['Month'] == start_month + i), state].empty else None
        for i in range(len(num_days_per_month))
    ]
    total_weighted_value = sum([value * days for value, days in zip(home_value_per_month, num_days_per_month) if not pd.isna(value)])
    weighted_home_value_during_fire = total_weighted_value / num_days_total
    total_impacted_home_value_estimate = weighted_home_value_during_fire * prim_threatened_aggregate
    return total_impacted_home_value_estimate
    
print(get_weighted_residential_value_for_fire('8/27/2005', '8/29/2005', 'California', 15)) # test on the first fire entry in the table

6800976.697397812


In [20]:
noaa_data_masked['weighted_home_value_during_fire'] = noaa_data_masked.apply(
    lambda row: get_weighted_residential_value_for_fire(
        row['start_date'],
        row['controlled_date'],
        abbreviation_to_state_name[row['state']],
        row['prim_threatened_aggregate']
    ),
    axis=1
)
states_ordered_by_devastation = noaa_data_masked.groupby('state').agg({'weighted_home_value_during_fire': 'sum'}).reset_index().sort_values(by='weighted_home_value_during_fire', ascending=False)
states_ordered_by_devastation[:3]

/var/folders/mt/lv8dl5zj78n8fw343s4t1jmc0000gn/T/ipykernel_70624/3749176147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noaa_data_masked['weighted_home_value_during_fire'] = noaa_data_masked.apply(


,state,weighted_home_value_during_fire
1,CA,3.811669e+11
3,ID,2.954208e+10
9,WA,2.361739e+10
